In [1]:
import numpy as np
import pandas as pd
from itertools import combinations_with_replacement
from scipy.stats import chi2
import scipy.stats as stats
ds = [206,99,103,105,104,489,108]

In [2]:
df = pd.read_excel('df.xlsx')

# Filter to just KHV rows
khv = df[df['Ethnic'] == 'KHV206g'] 

# Filter to just CYP2B6 rows
cyp2c9 = khv[khv['Gene'] == 'CYP2C9']

# Get unique CYP2B6 alleles 
cyp2c9_alleles = set(cyp2c9['Star Allele 1'].unique()) | set(cyp2c9['Star Allele 2'].unique())

print(cyp2c9_alleles)

{1, 2, 3, 5, 8, 9, 11, 13, 29}


In [3]:
cyp2c9_alleles = list(cyp2c9_alleles)
cyp2c9table = pd.DataFrame(cyp2c9_alleles, columns=['CYP2C9'])
cyp2c9table['KHV'] = 0.0
print(cyp2c9table)
print(cyp2c9_alleles)

   CYP2C9  KHV
0       1  0.0
1       2  0.0
2       3  0.0
3       5  0.0
4       8  0.0
5       9  0.0
6      11  0.0
7      13  0.0
8      29  0.0
[1, 2, 3, 5, 8, 9, 11, 13, 29]


In [4]:
khv_cyp2c9 = df.query("Ethnic == 'KHV206g' & Gene == 'CYP2C9'")

print(khv_cyp2c9)
print(cyp2c9_alleles)
start = 0
for allele in cyp2c9_alleles:
  res = 0
  for index, row in khv_cyp2c9.iterrows():
    if allele == row['Star Allele 1']:
      res += row['Percentage per allele']
    if allele == row['Star Allele 2']:  
      res += row['Percentage per allele']
  cyp2c9table.at[start,'KHV'] = res
  start += 1


to_drop = cyp2c9table[cyp2c9table['KHV'] < 1]

# Drop those rows by index
cyp2c9table.drop(index=to_drop.index, inplace=True)
# Divide by 100  
print(cyp2c9table)

      Ethnic             Alleles  Percentage    Gene  Star Allele 1  \
403  KHV206g   CYP2C9*1/CYP2C9*1     90.2900  CYP2C9              1   
404  KHV206g  CYP2C9*1/CYP2C9*11      0.4854  CYP2C9              1   
405  KHV206g  CYP2C9*1/CYP2C9*13      0.4854  CYP2C9              1   
406  KHV206g   CYP2C9*1/CYP2C9*2      1.4560  CYP2C9              1   
407  KHV206g   CYP2C9*1/CYP2C9*3      4.3690  CYP2C9              1   
408  KHV206g   CYP2C9*1/CYP2C9*8      0.4854  CYP2C9              1   
409  KHV206g   CYP2C9*1/CYP2C9*9      0.9709  CYP2C9              1   
410  KHV206g  CYP2C9*3/CYP2C9*29      0.4854  CYP2C9              3   
411  KHV206g   CYP2C9*3/CYP2C9*3      0.4854  CYP2C9              3   
412  KHV206g   CYP2C9*5/CYP2C9*8      0.4854  CYP2C9              5   

     Star Allele 2  Percentage per allele  
403              1               45.14500  
404             11                0.24270  
405             13                0.24270  
406              2                0.72800 

In [5]:
temp = cyp2c9table['CYP2C9'].values.tolist()
ethnics = df['Ethnic'].unique()
ethnics =np.delete(ethnics, -1)
value_to_delete = 'KHV'

# Find index of value 
idx = np.where(ethnics == value_to_delete)[0][0]
ethnics = np.delete(ethnics, idx)
print(ethnics)
cyp2c9_rows = df[df['Gene'] == 'CYP2C9']

# Create bool series to check if allele in temp
allele1_matches = cyp2c9_rows['Star Allele 1'].isin(temp)
allele2_matches = cyp2c9_rows['Star Allele 2'].isin(temp)

# Filter for rows where either allele matches 
mask = allele1_matches | allele2_matches
cyp2c9_population = cyp2c9_rows[mask]
cyp2c9_population = cyp2c9_population[cyp2c9_population['Ethnic'] != 'KHV206g']

print(cyp2c9_population)#print(temp)

['CEU' 'CHB' 'CHS' 'JPT' 'SAS' 'YRI']
    Ethnic             Alleles  Percentage    Gene  Star Allele 1  \
80     CEU   CYP2C9*1/CYP2C9*1     61.6200  CYP2C9              1   
81     CEU  CYP2C9*1/CYP2C9*12      1.0100  CYP2C9              1   
82     CEU   CYP2C9*1/CYP2C9*2     18.1800  CYP2C9              1   
83     CEU   CYP2C9*1/CYP2C9*3     10.1000  CYP2C9              1   
85     CEU   CYP2C9*2/CYP2C9*3      3.0300  CYP2C9              2   
87     CHB   CYP2C9*1/CYP2C9*1     89.3200  CYP2C9              1   
88     CHB  CYP2C9*1/CYP2C9*16      0.9709  CYP2C9              1   
89     CHB  CYP2C9*1/CYP2C9*29      0.9709  CYP2C9              1   
90     CHB   CYP2C9*1/CYP2C9*3      7.7670  CYP2C9              1   
91     CHB  CYP2C9*1/CYP2C9*33      0.9709  CYP2C9              1   
92     CHS   CYP2C9*1/CYP2C9*1     88.5700  CYP2C9              1   
93     CHS   CYP2C9*1/CYP2C9*2      0.9524  CYP2C9              1   
94     CHS  CYP2C9*1/CYP2C9*29      0.9524  CYP2C9              1

In [6]:
for ethnic in ethnics:
        cyp2c9table[ethnic] = 0.0

cyp2c9table = cyp2c9table.reset_index(drop=True)

print(cyp2c9table)

   CYP2C9       KHV  CEU  CHB  CHS  JPT  SAS  YRI
0       1  94.41605  0.0  0.0  0.0  0.0  0.0  0.0
1       3   2.91260  0.0  0.0  0.0  0.0  0.0  0.0


In [7]:
start = 0
for ethnic in ethnics:
    start = 0
    for starallele in temp:
        res = 0

        for index, row in cyp2c9_population.iterrows():
            if row['Ethnic'] == ethnic:
                if starallele == row['Star Allele 1']:
                    res += row['Percentage per allele']
                if starallele == row['Star Allele 2']:  
                    res += row['Percentage per allele']
                cyp2c9table.at[start,ethnic] = res
        #print(start)    
        start += 1
    cyp2c9table[ethnic] = cyp2c9table[ethnic].round(1)

cyp2c9table['KHV'] = cyp2c9table['KHV'].round(1)
cyp2c9table.to_csv('frq/cyp2c9.csv')
print(cyp2c9table)

   CYP2C9   KHV   CEU   CHB   CHS   JPT   SAS   YRI
0       1  94.4  76.3  94.7  93.8  97.6  82.6  77.8
1       3   2.9   6.6   3.9   4.8   1.9  10.8   0.0


In [8]:
init = {'CEU': [0], 'CHB': [0], 'CHS': [0], 'JPT': [0], 'SAS': [0], 'YRI': [0]}
gst = pd.DataFrame(init, dtype=float)

h_s_k = 1.0
for index,value in cyp2c9table.iterrows():
    h_s_k -= (cyp2c9table.at[index,'KHV']/100) * (cyp2c9table.at[index,'KHV']/100)

for idx in range(0,6):
    value = 0
    h_t = 1.0
    h_s = 1.0
    for id in range(0,2):
        h_t -= ((cyp2c9table.iloc[id,idx + 2] + cyp2c9table.iloc[id,1])/200) * ((cyp2c9table.iloc[id,idx + 2] + cyp2c9table.iloc[id,1])/200)
        h_s -= (cyp2c9table.iloc[id,idx + 2]/100) * (cyp2c9table.iloc[id,idx + 2]/100)
    h_s = (h_s + h_s_k)/2
    gst.iloc[0,idx] = (h_t - h_s)/h_t # You can modify the value here'''
gst = gst.round(4)
gst.index = ['CYP2C9']
gst.to_csv('gst/gst_cyp2c9.csv')
print(gst)

           CEU     CHB     CHS     JPT     SAS     YRI
CYP2C9  0.0317  0.0003  0.0009  0.0036  0.0238  0.0275


In [9]:
test = list(cyp2c9table['CYP2C9'])
test = ['CYP2C9*' + str(int(item)) for item in test]

def generate_gene_combinations(alleles):
    combinations = list(combinations_with_replacement(alleles, 2))
    sorted_combinations = ["/".join(sorted(comb)) for comb in combinations]
    return sorted_combinations

gene_combinations = generate_gene_combinations(test)
degrees_of_freedom = len(gene_combinations) - 1
print(gene_combinations)

['CYP2C9*1/CYP2C9*1', 'CYP2C9*1/CYP2C9*3', 'CYP2C9*3/CYP2C9*3']


In [10]:
observed = pd.DataFrame(gene_combinations, columns=['Gene'])
new_columns = ['KHV', 'CEU', 'CHB', 'CHS', 'JPT', 'SAS', 'YRI']
for column in new_columns:
    observed[column] = 0.0

print(observed)

                Gene  KHV  CEU  CHB  CHS  JPT  SAS  YRI
0  CYP2C9*1/CYP2C9*1  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1  CYP2C9*1/CYP2C9*3  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  CYP2C9*3/CYP2C9*3  0.0  0.0  0.0  0.0  0.0  0.0  0.0


In [11]:
l = len(khv_cyp2c9)
for x in range(0,l):
    if khv_cyp2c9.iloc[x,1] in gene_combinations:
        gene_to_update = khv_cyp2c9.iloc[x,1]
        new_value = khv_cyp2c9.iloc[x,2]
        observed.loc[observed['Gene'] == gene_to_update, 'KHV'] = new_value

length = len(cyp2c9_population)
for x in range(0,length):
    if cyp2c9_population.iloc[x,1] in gene_combinations:
        gene_to_update = cyp2c9_population.iloc[x,1]
        pop_to_update = cyp2c9_population.iloc[x,0]
        new_value = cyp2c9_population.iloc[x,2]
        observed.loc[observed['Gene'] == gene_to_update, pop_to_update ] = new_value 

for x in range(0,7):
    for y in range(0,len(gene_combinations)):
        observed.iloc[y,x+1] = observed.iloc[y,x+1] * ((ds[x] * 1.0)/ 100)

observed = observed.round(0)
print(observed)

                Gene    KHV   CEU   CHB   CHS   JPT    SAS   YRI
0  CYP2C9*1/CYP2C9*1  189.0  61.0  92.0  93.0  99.0  341.0  66.0
1  CYP2C9*1/CYP2C9*3   12.0  10.0   8.0   8.0   4.0   90.0   0.0
2  CYP2C9*3/CYP2C9*3    1.0   0.0   0.0   1.0   0.0    3.0   0.0


In [12]:
hypo = pd.DataFrame(gene_combinations, columns=['Gene'])
for column in new_columns:
    hypo[column] = 0.0

allele = cyp2c9table.iloc[:,0].tolist()

for x in range(0,7):
    temp = cyp2c9table.iloc[:,x + 1].tolist()
    for allele1 in range(0,len(temp)):
        for allele2 in range(allele1,len(temp)):
            if allele1 == allele2:
                new_value = (temp[allele1] / 100) * (temp[allele2] / 100)
            elif allele1 != allele2:
                new_value = 2 * (temp[allele1] / 100) * (temp[allele2] / 100)
            gene = f"CYP2C9*{int(allele[allele1])}/CYP2C9*{int(allele[allele2])}"
            row_index = hypo.index[hypo['Gene'] == gene].tolist()[0]
            hypo.iloc[row_index, x+1] = new_value

print(hypo)

                Gene       KHV       CEU       CHB       CHS       JPT  \
0  CYP2C9*1/CYP2C9*1  0.891136  0.582169  0.896809  0.879844  0.952576   
1  CYP2C9*1/CYP2C9*3  0.054752  0.100716  0.073866  0.090048  0.037088   
2  CYP2C9*3/CYP2C9*3  0.000841  0.004356  0.001521  0.002304  0.000361   

        SAS       YRI  
0  0.682276  0.605284  
1  0.178416  0.000000  
2  0.011664  0.000000  


In [13]:
for x in range(0,7):
    for y in range(0,len(gene_combinations)):
        hypo.iloc[y,x+1] = hypo.iloc[y,x+1] * ((ds[x] * 1.0))
        if hypo.iloc[y,x+1] < 1:
            hypo.iloc[y,x+1] = 1
hypo = hypo.round(0)
print(hypo)

                Gene    KHV   CEU   CHB   CHS   JPT    SAS   YRI
0  CYP2C9*1/CYP2C9*1  184.0  58.0  92.0  92.0  99.0  334.0  65.0
1  CYP2C9*1/CYP2C9*3   11.0  10.0   8.0   9.0   4.0   87.0   1.0
2  CYP2C9*3/CYP2C9*3    1.0   1.0   1.0   1.0   1.0    6.0   1.0


In [14]:
hihi = {"KHV": [0.0], "CEU": [0.0], "CHB": [0.0], "CHS": [0.0], "JPT": [0.0], "SAS": [0.0], "YRI": [0.0]}
chi_square = pd.DataFrame(hihi)
print(chi_square)
for x in range(0,7):
    res = 0
    for y in range(0,len(hypo)):
        chi_square_value = ((hypo.iloc[y,x+1] - observed.iloc[y,x+1]) * (hypo.iloc[y,x+1] - observed.iloc[y,x+1])) / (hypo.iloc[y,x+1])
        res += chi_square_value

    p_value = 1 - stats.chi2.cdf(res, degrees_of_freedom)
    chi_square.iloc[0,x] = p_value 

chi_square = chi_square.round(4)

chi_square.index = ['CYP2C9']
print(chi_square)

   KHV  CEU  CHB  CHS  JPT  SAS  YRI
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
           KHV     CEU     CHB     CHS     JPT     SAS     YRI
CYP2C9  0.8928  0.5613  0.6065  0.9408  0.6065  0.4168  0.3651
